Backups Since Last Full

In [ ]:
SET NOCOUNT ON;
SET ANSI_WARNINGS ON;
SET QUOTED_IDENTIFIER ON;
SET DATEFORMAT mdy;

DECLARE @sqlcmd NVARCHAR(max), @params NVARCHAR(600)
DECLARE @sqlmajorver int

SELECT @sqlmajorver = CONVERT(int, (@@microsoftversion / 0x1000000) & 0xff);

IF @sqlmajorver > 10
BEGIN
	SET @sqlcmd = N'SELECT ''Information'' AS [Category], ''Backups_since_last_Full'' AS [Information], 
[database_name] AS [Database_Name], CASE WHEN type = ''D'' THEN ''Database''
	WHEN type = ''I'' THEN ''Diff_Database''
	WHEN type = ''L'' THEN ''Log''
	WHEN type = ''F'' THEN ''File''
	WHEN type = ''G'' THEN ''Diff_file''
	WHEN type = ''P'' THEN ''Partial''
	WHEN type = ''Q'' THEN ''Diff_partial''
	ELSE NULL END AS [bck_type],
[backup_start_date], [backup_finish_date],
CONVERT(decimal(20,2),backup_size/1024.00/1024.00) AS [backup_size_MB],
CONVERT(decimal(20,2),compressed_backup_size/1024.00/1024.00) AS [compressed_backup_size_MB],
[recovery_model], [user_name],
database_backup_lsn AS [full_base_lsn], [differential_base_lsn], [expiration_date], 
[is_password_protected], [has_backup_checksums], [is_readonly], is_copy_only, [has_incomplete_metadata] AS [Tail_log]
FROM msdb.dbo.backupset bck1 (NOLOCK)
WHERE is_copy_only = 0 -- No COPY_ONLY backups
AND backup_start_date >= (SELECT MAX(backup_start_date) FROM msdb.dbo.backupset bck2 (NOLOCK) WHERE bck2.type IN (''D'',''F'',''P'') AND is_copy_only = 0 AND bck1.database_name = bck2.database_name)
ORDER BY database_name, backup_start_date DESC'
END
ELSE 
BEGIN
	SET @sqlcmd = N'SELECT ''Information'' AS [Category], ''Backups_since_last_Full'' AS [Information], 
[database_name] AS [Database_Name], CASE WHEN type = ''D'' THEN ''Database''
	WHEN type = ''I'' THEN ''Diff_Database''
	WHEN type = ''L'' THEN ''Log''
	WHEN type = ''F'' THEN ''File''
	WHEN type = ''G'' THEN ''Diff_file''
	WHEN type = ''P'' THEN ''Partial''
	WHEN type = ''Q'' THEN ''Diff_partial''
	ELSE NULL END AS [bck_type],
[backup_start_date], [backup_finish_date], 
CONVERT(decimal(20,2),backup_size/1024.00/1024.00) AS [backup_size_MB],
''NA'' AS [compressed_backup_size_MB], 
[recovery_model], [user_name],
database_backup_lsn AS [full_base_lsn], [differential_base_lsn], [expiration_date], 
[is_password_protected], [has_backup_checksums], [is_readonly], is_copy_only, [has_incomplete_metadata] AS [Tail_log]
FROM msdb.dbo.backupset bck1 (NOLOCK)
WHERE is_copy_only = 0 -- No COPY_ONLY backups
AND backup_start_date >= (SELECT MAX(backup_start_date) FROM msdb.dbo.backupset bck2 (NOLOCK) WHERE bck2.type IN (''D'',''F'',''P'') AND is_copy_only = 0 AND bck1.database_name = bck2.database_name)
ORDER BY database_name, backup_start_date DESC'
END;

EXECUTE sp_executesql @sqlcmd;